In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import scipy.io
from scipy.optimize import curve_fit
from IPython.display import clear_output

In [ ]:
eta_base = scipy.io.loadmat('eta_base.mat')['eta_save']
eta_base.shape

In [ ]:
eta_comp = scipy.io.loadmat('eta_perturbed.mat')['eta_save']
eta_comp.shape

Finding $|d|$ time series by summing absolute values of bed elevation differences at each node

$ |d(t)|=  \sum_{j=1}^6  |\eta^{pert}_j - \eta^{base}_j |$

In [ ]:
d = []
for t in range(eta_comp.shape[1]):
    d_j = 0
    for j in range(6):
        d_j += np.abs(eta_comp[j+1][t] - eta_base[j+1][t])
    d.append(d_j)

d = np.array(d)

In [ ]:
times = scipy.io.loadmat('eta_base.mat')['t_save'][0]
times.shape

In [ ]:
plt.plot(times, d)

## Curve Fitting



In [ ]:
# General exponential function
def f(t,k, d_0, t_0):
    return d_0*np.exp(k*(t-t_0))

### Finding chaotic window

Iterates over a range of reasonable lower and upper bound combinations, curve fits, and selects the one with the best $r^2$.

In [ ]:
lower_bounds = np.linspace(0, 1000, 10)
upper_bounds = np.linspace(3000,4000,10)

best_lower_bound = 0
best_upper_bound= 4000
best_bounds_r_squared = 0

for lower_bound in lower_bounds:
    for upper_bound in upper_bounds:
        
        lower_bound = int(lower_bound)
        upper_bound = int(upper_bound)
        
        if upper_bound > lower_bound:
            d_truncated = np.copy(d)[lower_bound:upper_bound]
            times_truncated = np.copy(times)[lower_bound:upper_bound]
            
            popt, pcov = curve_fit(lambda t, b: d_truncated[0] * np.exp(b * (t-times_truncated[0])),  times_truncated, d_truncated)

            residuals = np.sum((d_truncated - f(times_truncated, popt[0], d_truncated[0], times_truncated[0]))**2)

            sum_squares = np.sum((d_truncated - np.mean(d_truncated))**2)

            r_squared = 1 - (residuals / sum_squares)

            if 1-r_squared < 1-best_bounds_r_squared:
                best_bounds_r_squared = r_squared
                best_lower_bound = lower_bound
                best_upper_bound = upper_bound
                
                
        
d = d[best_lower_bound : best_upper_bound]
times = times[best_lower_bound : best_upper_bound]

In [ ]:
popt, pcov = curve_fit(lambda t, b: d[0] * np.exp(b * (t-times[0])), times, d)

In [ ]:
residuals = np.sum((d- f(times,popt[0], d[0], times[0]))**2)

sum_squares = np.sum((d-np.mean(d))**2)

r_squared = 1 - (residuals / sum_squares)

In [ ]:
plt.figure()
plt.plot(times, d)
plt.plot(times, f(times, popt[0], d[0], times[0]))
plt.xlabel('t')
plt.ylabel('$|d|$')
plt.legend(['$|d|$', '$y = d_0 e^{\lambda t}$, $\lambda = $' + str(np.round(popt[0], 2)) + ', $r^2$ = ' + str(np.round(r_squared,2))])

# Plotting

In [ ]:
# create grid for different subplots
spec = gridspec.GridSpec(ncols=1, nrows=2, height_ratios=[2, 1])

fig = plt.figure(figsize=(14, 18))
ax = fig.add_subplot(spec[0], projection='3d')

# Position nodes in a cantor set in a 2D plane (manually for 7 nodes)
## this would be a fun one to automate
nodes = {
    "location": [1/2,0],
    "children": [
            {
                "location": [1/6, 1/2],
                "children": [
                    {
                        "location": [1/18, 1]
                    },
                    {
                        "location": [4/18, 1]
                    }
                ]
            },
            {
                "location": [5/6, 1/2],
                "children": [
                    {
                        "location": [14/18, 1]
                    },
                    {
                        "location": [17/18, 1]
                    }
                ]
            }
     ]
}

def plot_nodes(node, color):
    if 'children' in node:
        for i in range(len(node['children'])):
            child = node['children'][i]
            ax.plot([node['location'][0], child['location'][0]], [node['location'][1], child['location'][1]], [node['location'][2], child['location'][2]], color)
            plot_nodes(child, color)
    

In [ ]:
bed_elevations = scipy.io.loadmat('eta_base.mat')['eta_save']
bed_elevations_2 = scipy.io.loadmat('eta_perturbed.mat')['eta_save']
frames = []
frames_2 = []
for i in range(bed_elevations.shape[1]):
    bed_elevation = bed_elevations[:, i]
    frames.append(
        {
            "location": [1/2,0, bed_elevation[0]],
            "children": [
                    {
                        "location": [1/6, 1/2, bed_elevation[1]],
                        "children": [
                            {
                                "location": [1/18, 1, bed_elevation[3]]
                            },
                            {
                                "location": [4/18, 1, bed_elevation[4]]
                            }
                        ]
                    },
                    {
                        "location": [5/6, 1/2, bed_elevation[2]],
                        "children": [
                            {
                                "location": [14/18, 1, bed_elevation[5]]
                            },
                            {
                                "location": [17/18, 1, bed_elevation[6]]
                            }
                        ]
                    }
             ]
        }
    )
for i in range(bed_elevations.shape[1]):
    bed_elevation = bed_elevations_2[:, i]
    frames_2.append(
        {
            "location": [1/2,0, bed_elevation[0]],
            "children": [
                    {
                        "location": [1/6, 1/2, bed_elevation[1]],
                        "children": [
                            {
                                "location": [1/18, 1, bed_elevation[3]]
                            },
                            {
                                "location": [4/18, 1, bed_elevation[4]]
                            }
                        ]
                    },
                    {
                        "location": [5/6, 1/2, bed_elevation[2]],
                        "children": [
                            {
                                "location": [14/18, 1, bed_elevation[5]]
                            },
                            {
                                "location": [17/18, 1, bed_elevation[6]]
                            }
                        ]
                    }
             ]
        }
    )


In [ ]:
for i in list(np.linspace(best_lower_bound, best_upper_bound, 500)):
    i = int(i)
    fig = plt.figure(figsize=(14, 18))
    ax = fig.add_subplot(spec[0], projection='3d')
    ax.view_init(elev=20., azim=45, roll=0)
    ax.set_zlim(12, 18)
    ax.set_xlabel('Nodes spaced arbitrarily in $x$ and $y$ for visual clarity')
    ax.set_zlabel('Bed Elevation $\eta$ (meters)')
    plot_nodes(frames[i], 'b')
    plot_nodes(frames_2[i], 'r')
   
    # Trasverse both trees at once and plot segements between nodes
    def plot_diff(frame, frame_2):
        ax.plot([frame['location'][0], frame_2['location'][0]], [frame['location'][1], frame_2['location'][1]], [frame['location'][2], frame_2['location'][2]], color='limegreen')
        if 'children' in frame.keys():
            for j in range(len(frame['children'])):
                plot_diff(frame['children'][j], frame_2['children'][j])
        
    plot_diff(frames[i], frames_2[i])
    
    ax1 = fig.add_subplot(spec[1])
    ax1.plot(np.copy(times)[:i-best_lower_bound], np.copy(d)[:i-best_lower_bound], color='limegreen')
    ax1.plot(np.copy(times)[:i-best_lower_bound], f(np.copy(times)[:i-best_lower_bound], popt[0], d[0], times[0]))
    ax1.set_xlabel('time (years)')
    ax1.set_xlim(times[0], 1.25* times[-1])
    ax1.set_ylim(0, 1.25 * np.max(d))
    ax1.set_ylabel('$|d| (meters)$')
    ax1.legend(['$|d|$', '$y = d_0 e^{\lambda t}$, $\lambda = $' + str(np.round(popt[0], 2)) + ', $r^2$ = ' + str(np.round(r_squared,2))])
    
    plt.savefig(f'frames/{str(i).zfill(5)}.png')
    clear_output(wait=True)